# Defining Project Path

In [13]:
import sys
from pathlib import Path
import os
sys.path.append(str(Path(os.getcwd()).resolve().parent))

# Path of the project
PROJECT_PATH = Path(os.getcwd()).resolve().parent

# Loading Model from checkpoint

In [3]:
from ultralytics import YOLO
from huggingface_hub import hf_hub_download

pt_path = hf_hub_download(
    repo_id="Mikolaj1234/yolov11m-football-ai",
    filename="best.pt"
)

MODEL = YOLO(pt_path)
MODEL.to("cuda")

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_

In [64]:
import supervision as sv
from tqdm import tqdm

VIDEO_PATH = PROJECT_PATH / 'data' / 'origin_videos' / 'football_06.mp4'
video_info = sv.VideoInfo.from_video_path(VIDEO_PATH)

PROCESSED_VIDEO_DIR = PROJECT_PATH / 'processed_video'
PROCESSED_VIDEO_DIR.mkdir(parents=True, exist_ok=True)
TARGET_VIDEO_PATH = PROCESSED_VIDEO_DIR / f'{VIDEO_PATH.stem}_yolov11m_processed.mp4'
video_sink = sv.VideoSink(TARGET_VIDEO_PATH, video_info=video_info)

# Define annotator objects for visualization
box_annotator = sv.BoxAnnotator(
    color=sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000')
)
id2label = {
    0: 'Ball',
    1: 'Goalkeeper',
    2: 'Player',
    3: 'Referee'
}
label2id = {v: k for k, v in id2label.items()}

tracker = sv.ByteTrack()
tracker.reset()

frame_generator = sv.get_video_frames_generator(VIDEO_PATH)
with video_sink:
    for frame in tqdm(frame_generator, total=video_info.total_frames, desc=f'Processing video: {VIDEO_PATH.stem}'):
        outputs = MODEL.predict(frame, verbose=False)[0]

        bboxes = outputs.boxes.xyxy.cpu().numpy()
        cls_ids = outputs.boxes.cls.cpu().numpy().astype(int)
        scores = outputs.boxes.conf.cpu().numpy()
        detections = sv.Detections(
            xyxy=bboxes,
            confidence=scores,
            class_id=cls_ids
        )

        # detections = detections[detections.confidence > 0.3]
        detections = detections.with_nms(threshold=0.3, class_agnostic=True)

        ball_detections = detections[detections.class_id == label2id['Ball']]
        ball_detections.xyxy = sv.pad_boxes(xyxy=ball_detections.xyxy, px=10)

        all_detections = detections[detections.class_id != label2id['Ball']]
        all_detections = tracker.update_with_detections(detections=all_detections)

        goalkeepers_detections = all_detections[all_detections.class_id == label2id['Goalkeeper']]
        players_detections = all_detections[all_detections.class_id == label2id['Player']]
        referees_detections = all_detections[all_detections.class_id == label2id['Referee']]

        all_detections = sv.Detections.merge([players_detections, goalkeepers_detections, referees_detections])

        labels = [
            f"#{id2label[cls_id]}"
            for cls_id
            in all_detections.class_id
        ]
        
        annotated_img = frame.copy()
        annotated_img = box_annotator.annotate(annotated_img, all_detections)
        annotated_img = label_annotator.annotate(scene=annotated_img, detections=all_detections, labels=labels)
        annotated_img = box_annotator.annotate(scene=annotated_img, detections=ball_detections)

        video_sink.write_frame(annotated_img)

Processing video: football_06: 100%|██████████| 750/750 [00:27<00:00, 26.90it/s]
